<a href="https://colab.research.google.com/github/DhruvAjayToshniwal/Stock-Trading-Backtest-using-SMA/blob/main/Stock_trading_Backtest_using_SMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance alpaca-trade-api

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

def get_daily_data(symbols):
    data = yf.download(symbols, start='2020-01-01', group_by='ticker')  # get data from the start of 2020
    return {symbol: data[symbol] for symbol in symbols}

# Example usage:
data = get_daily_data(['AAPL', 'GOOG', 'MSFT'])
print(data)


[*********************100%***********************]  3 of 3 completed
{'AAPL':                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-01-02   74.059998   75.150002   73.797501   75.087502   73.347923   
2020-01-03   74.287498   75.144997   74.125000   74.357498   72.634857   
2020-01-06   73.447502   74.989998   73.187500   74.949997   73.213615   
2020-01-07   74.959999   75.224998   74.370003   74.597504   72.869293   
2020-01-08   74.290001   76.110001   74.290001   75.797501   74.041481   
...                ...         ...         ...         ...         ...   
2023-07-10  189.259995  189.990005  187.039993  188.610001  188.610001   
2023-07-11  189.160004  189.300003  186.600006  188.080002  188.080002   
2023-07-12  189.679993  191.699997  188.470001  189.770004  189.770004   
2023-07-13  190.500000  191.190002  189.779999  190.539993  190.539993   
2023-07-14  190.229996  191.179993

In [ ]:
def calculate_moving_averages(data, short_window=20, long_window=50):
    signals = pd.DataFrame(index=data.index)
    signals['signal'] = 0.0

    # Create short simple moving average over the short window
    signals['short_mavg'] = data['Close'].rolling(window=short_window, min_periods=1, center=False).mean()

    # Create long simple moving average over the long window
    signals['long_mavg'] = data['Close'].rolling(window=long_window, min_periods=1, center=False).mean()

    # Create signals
    signals['signal'][short_window:] = np.where(signals['short_mavg'][short_window:] > signals['long_mavg'][short_window:], 1.0, -1.0)

    # Generate trading orders
    signals['positions'] = signals['signal'].diff()

    return signals


In [ ]:
def backtest(data, signals, initial_capital=100000.0):
    positions = pd.DataFrame(index=signals.index).fillna(0.0)
    positions['Position'] = 100 * signals['signal']

    portfolio = positions.multiply(data['Adj Close'], axis=0)
    pos_diff = positions.diff()

    portfolio['holdings'] = (positions.multiply(data['Adj Close'], axis=0)).sum(axis=1)
    portfolio['cash'] = initial_capital - (pos_diff.multiply(data['Adj Close'], axis=0)).sum(axis=1).cumsum()

    portfolio['total'] = portfolio['cash'] + portfolio['holdings']
    portfolio['returns'] = portfolio['total'].pct_change()

    return portfolio

In [ ]:
def place_trades(api, signals, symbol):
    print(f"Latest signal: {signals['signal'][-1]}")
    order = None  # Ensure order is defined in all code paths

    if signals['signal'][-1] == 1.0:  # check if the latest signal is a buy signal
        print("Submitting buy order")
        order = api.submit_order(
            symbol=symbol,
            qty=100,
            side='buy',
            type='market',
            time_in_force='gtc'
        )
        print(order)
    elif signals['signal'][-1] == -1.0:  # check if the latest signal is a sell signal
        print("Submitting sell order")
        order = api.submit_order(
            symbol=symbol,
            qty=100,
            side='sell',
            type='market',
            time_in_force='gtc'
        )
        print(order)

    if order is not None:
        return order.id


In [ ]:
def check_order(api, order_id):
    order = api.get_order(order_id)
    print(f"Order status: {order.status}")
    return order.status

In [ ]:
def plot_data(data, signals, portfolio, symbol):
    fig = go.Figure()

    # Plot the adjusted close price, short-term and long-term moving averages
    fig.add_trace(go.Scatter(x=data.index, y=data['Adj Close'], mode='lines', name='Adj Close'))
    fig.add_trace(go.Scatter(x=data.index, y=signals['short_mavg'], mode='lines', name='Short Mavg'))
    fig.add_trace(go.Scatter(x=data.index, y=signals['long_mavg'], mode='lines', name='Long Mavg'))

    fig.layout.update(title_text=f'{symbol} price and moving averages', xaxis_rangeslider_visible=True)
    fig.show()

    fig2 = go.Figure()

    # Plot the total value of the portfolio
    fig2.add_trace(go.Scatter(x=portfolio.index, y=portfolio['total'], mode='lines', name='Total Value'))

    fig2.layout.update(title_text=f'{symbol} Portfolio Value', xaxis_rangeslider_visible=True)
    fig2.show()

In [ ]:
#use your alpaca paper trade api
api = tradeapi.REST('api key', 'secret', base_url='https://paper-api.alpaca.markets')

# Fetch data for all symbols
stock_data = get_daily_data(['AAPL', 'GOOG', 'MSFT'])

# Calculate signals and backtest for each symbol
for symbol in ['AAPL', 'GOOG', 'MSFT']:
    data = stock_data.get(symbol)
    if data is None:
        print(f"No data found for symbol: {symbol}")
        continue

    signals = calculate_moving_averages(data)
    print(f"{symbol} signals:")
    print(signals)

    portfolio = backtest(data, signals)  # Pass the data for the symbol
    print(f"{symbol} portfolio:")
    print(portfolio)

    order_id = place_trades(api, signals, symbol)  # Store the order id
    if order_id is not None:
            check_order(api, order_id)  # Check the order status

    plot_data(data, signals, portfolio, symbol)  # Create plots

[*********************100%***********************]  3 of 3 completed
AAPL signals:
            signal  short_mavg   long_mavg  positions
Date                                                 
2020-01-02     0.0   75.087502   75.087502        NaN
2020-01-03     0.0   74.722500   74.722500        0.0
2020-01-06     0.0   74.798332   74.798332        0.0
2020-01-07     0.0   74.748125   74.748125        0.0
2020-01-08     0.0   74.958000   74.958000        0.0
...            ...         ...         ...        ...
2023-07-10     1.0  187.330999  179.212600        0.0
2023-07-11     1.0  187.686999  179.606000        0.0
2023-07-12     1.0  187.985999  180.007800        0.0
2023-07-13     1.0  188.347499  180.426800        0.0
2023-07-14     1.0  188.684499  180.869800        0.0

[889 rows x 4 columns]
AAPL portfolio:
                Position      holdings           cash          total   returns
Date                                                                          
2020-01-02      0

GOOG signals:
            signal  short_mavg   long_mavg  positions
Date                                                 
2020-01-02     0.0   68.368500   68.368500        NaN
2020-01-03     0.0   68.200748   68.200748        0.0
2020-01-06     0.0   68.704000   68.704000        0.0
2020-01-07     0.0   68.944750   68.944750        0.0
2020-01-08     0.0   69.199001   69.199001        0.0
...            ...         ...         ...        ...
2023-07-10     1.0  121.958500  119.478300        0.0
2023-07-11     1.0  121.700500  119.665100        0.0
2023-07-12     1.0  121.464000  119.893100        0.0
2023-07-13     1.0  121.484000  120.235500        0.0
2023-07-14     1.0  121.550000  120.629900        0.0

[889 rows x 4 columns]
GOOG portfolio:
                Position      holdings           cash          total   returns
Date                                                                          
2020-01-02      0.000000      0.000000  100000.000000  100000.000000       NaN
2020-01

MSFT signals:
            signal  short_mavg   long_mavg  positions
Date                                                 
2020-01-02     0.0  160.619995  160.619995        NaN
2020-01-03     0.0  159.619995  159.619995        0.0
2020-01-06     0.0  159.423330  159.423330        0.0
2020-01-07     0.0  158.962498  158.962498        0.0
2020-01-08     0.0  159.187997  159.187997        0.0
...            ...         ...         ...        ...
2023-07-10     1.0  336.405498  324.913398        0.0
2023-07-11     1.0  336.689497  325.466199        0.0
2023-07-12     1.0  336.956998  326.064999        0.0
2023-07-13     1.0  337.375497  326.806999        0.0
2023-07-14     1.0  337.770497  327.603599        0.0

[889 rows x 4 columns]
MSFT portfolio:
                Position      holdings           cash          total   returns
Date                                                                          
2020-01-02      0.000000      0.000000  100000.000000  100000.000000       NaN
2020-01